**SCRAPING: Sociale firma's**

Install packages if not already installed

In [ ]:
import sys
!{sys.executable} -m pip install selenium
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install beautifulsoup4

^C


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
from bs4 import BeautifulSoup
import re


**SOCIALE ZAKEN**

In [ ]:
# Initialize the webdriver
driver = webdriver.Firefox()  # or use your preferred browser
wait = WebDriverWait(driver, 10)

# Open the page
driver.get("https://socialezaken.info/nl/gemeente")
time.sleep(3)  # allow page to load

In [ ]:
results = []

# get all cards (by class "innah")
cards = wait.until(EC.presence_of_all_elements_located(
    (By.CSS_SELECTOR, "div.innah")
))

# Use indexes for stability
for i in range(len(cards)):

    # re-find cards on every loop
    cards = wait.until(EC.presence_of_all_elements_located(
        (By.CSS_SELECTOR, "div.innah")
    ))
    card = cards[i]

    # extract name inside h2
    try:
        h2 = card.find_element(By.TAG_NAME, "h2")
        name = h2.text.strip()
    except:
        name = ""
    
    # click card
    driver.execute_script("arguments[0].scrollIntoView(true);", card)
    time.sleep(0.5)
    card.click()

    # extract address on detail page
    try:
        sticky = wait.until(EC.presence_of_element_located(
            (By.CSS_SELECTOR, "div.four.sticky")
        ))
        p = sticky.find_element(By.CSS_SELECTOR, "p.zero-bottom")
        address = p.text.strip()
        address = address.replace("\n", ", ")
        
    except:
        address = ""
    
    # extract category
    try:
        category_element = driver.find_element(By.CSS_SELECTOR, "ul.zero-horizontal.filter.clearfix li span")
        category = category_element.text.strip()
    except:
        category = ""

    # extract impact label
    try:
        impact_element = driver.find_element(By.CSS_SELECTOR, "ul.filter.clearfix:not(.zero-horizontal) li")
        impact_label = impact_element.text.strip()
    except:
        impact_label = ""

    # save result
    results.append({
        "Naam": name,
        "Adres": address,
        "Categorie": category,
        "Impact": impact_label
    })

    # go back to previous page
    driver.back()
    time.sleep(1)

# Convert to DataFrame
socialezaken = pd.DataFrame(results)
driver.quit()

WebDriverException: Message: Failed to decode response from marionette


**SOCIAL ENTERPRISE NL**

In [ ]:
driver = webdriver.Firefox()
driver.get("https://social-enterprise.nl/onze-leden/leden-kaart/")

# Let the page load
driver.implicitly_wait(5)

In [ ]:
# Get page source
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

results = []

# All <span class="full-search-data">
for span in soup.find_all("span", class_="full-search-data"):
    text = span.get_text(strip=True)
    # Split address and name at the last hyphen
    if "-" in text:
        address, name = map(str.strip, text.rsplit("-", 1))
    else:
        address, name = "", text
    # Extract postal code (four digits at start of city line)
    postal_match = re.search(r"\b(\d{4})\b", address)
    if postal_match:
        postal = int(postal_match.group(1))
        if 1000 <= postal <= 1109:  # Amsterdam postal code range
            results.append({
                "Naam": name,
                "Adres": address.replace("\n", ", ")
            })

# Create DataFrame
socialenterprisenl = pd.DataFrame(results)
print(socialenterprisenl)

driver.quit()

                    Naam                                          Adres
0                Droppie  Van Limburg Stirumplein 1 , 1051 BD Amsterdam
1      Accent Schoonmaak          Karl Grogersingel , 1069 NX Amsterdam
2               twinpacT                   Kabelweg , 1014 BB Amsterdam
3                TolkApp             Haparandadam 7 , 1013 AK Amsterdam
4                  Dappr              Stromarkt 4 D , 1012 SW Amsterdam
..                   ...                                            ...
98              Sumthing            Krugerplein 36A , 1091 LA Amsterdam
99             Blue Road            herengracht 495 , 1017 BT Amsterdam
100         BORO*ATELIER   Ferdinand Huyckstraat 26 , 1061 HW Amsterdam
101  LemonAid & ChariTea             Jollemanhof 17 , 1019 GW Amsterdam
102          Talentcoach  Weth In `t Veldstraat 120 , 1107 BL Amsterdam

[103 rows x 2 columns]


**BCORP**
B corp: eerst de dataset in csv formaat downloaden van https://kb.bimpactassessment.net/en/support/solutions/articles/43000570530-using-sql-in-b-corp-impact-data-tutorial-

Vervolgens de path (file locatie) aanpassen in de volgende cell (In plaats van "Data/B-Corp.csv")

In [ ]:
Bcorp = pd.read_csv(r"C:\Users\engele016\OneDrive - Gemeente Amsterdam\Documenten\Github\Impact-data-scraper\B Corp Impact Data.csv")

C:\Users\engele016\AppData\Local\Temp\ipykernel_5740\1910107611.py:1: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  Bcorp = pd.read_csv(r"C:\Users\engele016\OneDrive - Gemeente Amsterdam\Documenten\Github\Impact-data-scraper\B Corp Impact Data.csv")


In [ ]:
Bcorp = Bcorp[["company_name","date_first_certified", "date_certified", "current_status","website", "b_corp_profile", "sector", "city", "country", "other_countries_of_operation" ]]

In [ ]:
filtered = Bcorp[
    (Bcorp["city"] == "Amsterdam") |
    (Bcorp["other_countries_of_operation"] == "The Netherlands")|
    (Bcorp["city"] == "AMSTERDAM")]

In [ ]:
filtered = filtered.drop_duplicates(subset="company_name", keep="first")

In [ ]:
filtered.shape

(202, 10)

In [ ]:
bcorp_old = pd.read_excel(r"C:\Users\engele016\OneDrive - Gemeente Amsterdam\Documenten\Github\Impact-data-scraper\bcorp en locatie isolated.xlsx")

In [ ]:

filtered = filtered.rename(columns={"company_name": "Naam"})

In [ ]:
# filtered: has at least ["Naam", "current_status"] where current_status is "certified" or "de-certified"
# bcorp_old: has at least ["Naam", "Adres"]

def _norm_name(s: pd.Series) -> pd.Series:
    return (s.astype("string")
              .str.strip()
              .str.replace(r"\s+", " ", regex=True)
              .str.casefold())

# ---- 1) Copies + normalized join keys ----
f = filtered.copy()
b = bcorp_old.copy()

f["_key"] = _norm_name(f["Naam"])
b["_key"] = _norm_name(b["Naam"])

# ---- 2) If filtered has duplicates per company, keep one (edit if you have a date column) ----
f = (f.dropna(subset=["_key"])
       .drop_duplicates(subset=["_key"], keep="last"))

# ---- 3) Full outer merge to keep companies from BOTH tables ----
new_df = b.merge(
    f[["_key", "current_status"]],
    on="_key",
    how="outer"
)

# ---- 4) Build Naam: prefer bcorp_old name, otherwise fall back to filtered name ----
f_name_map = f.drop_duplicates("_key").set_index("_key")["Naam"]
new_df["Naam"] = new_df["Naam"].combine_first(
    new_df["_key"].map(f_name_map)
)


# ---- 6) Status fill for companies not in filtered yet ----
# If you prefer unknown instead, change "certified" -> "unknown"
new_df["current_status"] = new_df["current_status"].fillna("certified")

# ---- 7) Flag companies that are NOT decertified ----
new_df["not_decertified"] = new_df["current_status"].ne("de-certified")

# ---- 8) Count empty / None address rows (None, NaN, or blank/whitespace) ----
addr = new_df["address"].astype("string")
empty_or_none_count = int(new_df["address"].isna().sum() + addr.str.strip().eq("").sum())

print("Empty/None address rows:", empty_or_none_count)

# ---- 9) Optional cleanup/order ----
bcorp = new_df.drop(columns=["_key"])  # keep "Adres" if you want; otherwise drop it too
# new_df = new_df.drop(columns=["_key", "Adres"])

cols_first = ["Naam", "current_status", "not_decertified", "address"]
bcorp = bcorp[cols_first + [c for c in bcorp.columns if c not in cols_first]]


Empty/None address rows: 68


,company_name,current_status,not_decertified,address,Type organisatie,Adres,Postcode,Stadsdeel
0,10DAYS,certified,True,Cornelis Schuytstraat 18,Bcorp,Cornelis Schuytstraat 18,NaN,Amsterdam Zuid
1,A Beautiful Story B.V.,certified,True,NaN,Bcorp,NaN,NaN,Amsterdam Oost
2,A-dam Underwear B.V.,certified,True,Danzigerbocht,Bcorp,Danzigerbocht,NaN,Amsterdam Westpoort
3,Academica University of Applied Sciences,certified,True,Weteringschans 28,Bcorp,Weteringschans 28,NaN,Amsterdam Centrum
4,Ace & Tate,certified,True,Singel 258,B-Corp,Singel 258,1016 AB,Amsterdam Centrum
...,...,...,...,...,...,...,...,...
211,Young Colfield,certified,True,Hobaostraat 12,Bcorp,Hobaostraat 12,NaN,Amsterdam Zuidoost
212,Yumeko,certified,True,x,Bcorp,x,NaN,Amsterdam Centrum
213,ZO Clean Planet BV,certified,True,Tweede Oosterparkstraat,Bcorp,Tweede Oosterparkstraat,NaN,Amsterdam oost
214,Zoku and Beyond BV,certified,True,Nieuwe Kerkstraat 104,Bcorp,Nieuwe Kerkstraat 104,NaN,Amsterdam Centrum


In [ ]:
# drop all "current_status" == "de-certified" and keep only "certified" 
bcorp = bcorp[bcorp["current_status"] == "certified"]


In [ ]:
bcorp.shape

(182, 8)

**STEWARD OWNERSHIP**

In [ ]:
URL="https://wearestewards.nl/en/companies/"
LIST="div.mU4ghb-pbTTYe-n0tgWb-haAclf"
ROW="div.HzV7m-pbTTYe-ibnC6b.pbTTYe-ibnC6b-d6wfac"
MORE="div.HzV7m-pbTTYe-bN97Pc-ti6hGc-z5C9Gb-haAclf"
BACK="span.HzV7m-tJHJj-LgbsSe-Bz112c.qqvbed-a4fUwd-LgbsSe-Bz112c"

d=webdriver.Firefox(); w=WebDriverWait(d,20); d.get(URL); time.sleep(5)
R=[]; S=set()
PC = re.compile(r"\b\d{4}\s?[A-Z]{2}\b", re.I)

for fr in d.find_elements(By.CSS_SELECTOR,"iframe"):
    s=(fr.get_attribute("src") or "").lower()
    if "google" in s and "maps" in s: d.switch_to.frame(fr); break

w.until(EC.presence_of_element_located((By.CSS_SELECTOR, LIST)))

stall=0; last=0
while stall<5:
    ms=d.find_elements(By.CSS_SELECTOR, MORE)
    if ms and ms[0].is_displayed():
        d.execute_script("arguments[0].scrollIntoView({block:'center'});arguments[0].click();", ms[0]); time.sleep(.6)
    c=len(d.find_elements(By.CSS_SELECTOR, ROW))
    stall = stall+1 if c<=last else 0; last=c
    box=d.find_element(By.CSS_SELECTOR, LIST)
    d.execute_script("arguments[0].scrollTop+=arguments[0].clientHeight;", box); time.sleep(.3)

i=0; stall=0; last=0
while stall<5:
    rows=d.find_elements(By.CSS_SELECTOR, ROW)
    while i < len(rows):
        rows=d.find_elements(By.CSS_SELECTOR, ROW)
        r=rows[i]; i+=1
        d.execute_script("arguments[0].scrollIntoView({block:'center'});arguments[0].click();", r); time.sleep(.6)

       # name (reliable)
        try:
            name = w.until(lambda _:
                (next(((e.get_attribute("textContent") or "").strip()
                    for e in d.find_elements(By.CSS_SELECTOR,"div.qqvbed-tJHJj-fmcmS")
                    if (e.get_attribute("textContent") or "").strip()), ""))
            )
        except:
            name = ""


        # address from visible text (postcode line)
        txt=[l.strip() for l in d.find_element(By.TAG_NAME,"body").text.splitlines() if l.strip()]
        cand=[l for l in txt if PC.search(l)]
        addr=""
        for l in cand:
            if any(c.isdigit() for c in l) and "," in l: addr=l; break
        if not addr and cand:
            j=txt.index(cand[0]); addr=(txt[j-1]+", "+txt[j]) if j>0 else txt[j]
        addr=re.sub(r"\s+"," ",addr).strip()

        if addr and (name,addr) not in S:
            S.add((name,addr)); R.append({"Naam":name,"Adres":addr})

        w.until(EC.element_to_be_clickable((By.CSS_SELECTOR, BACK))).click()
        w.until(EC.presence_of_element_located((By.CSS_SELECTOR, LIST))); time.sleep(.2)

    box=d.find_element(By.CSS_SELECTOR, LIST)
    d.execute_script("arguments[0].scrollTop+=arguments[0].clientHeight*0.9;", box); time.sleep(.6)
    c=len(d.find_elements(By.CSS_SELECTOR, ROW))
    stall = stall+1 if c<=last else 0; last=c

stewardownership=pd.DataFrame(R)
#filter on amsterdam
stewardownership = stewardownership[stewardownership["Adres"].str.contains(r"\bAmsterdam\b", case=False, na=False)]
d.quit()


      Naam                                           Adres
0   Aarden              Sonsbeeksingel 147, 6822 BL Arnhem
1  Alkemio  Wilhelmina van Pruisenweg 35, 2595 AN Den Haag
Amsterdam only:
Empty DataFrame
Columns: [Naam, Adres]
Index: []


**CODE SOCIALE ONDERNEMINGEN**
Omdat de filter functie voor regio op de website van code sociale ondernemingen niet accuraat werkt moet er een script geschreven worden dat de gehele longlist door parsed en per card checkt of het bedrijf in amsterdam gelokeerd is.

In [ ]:
URL="https://codesocialeondernemingen.nl/het-register/onze-deelnemers/"
CARD="a.kb-section-link-overlay"
DETAIL_H1="h1.kt-adv-heading174_0a478e-5a.wp-block-kadence-advancedheading"
AMSTERDAM_P="p.kt-adv-heading174_24656e-d7.wp-block-kadence-advancedheading"
NEXT="a.next.page-numbers"

driver=webdriver.Firefox()
wait=WebDriverWait(driver, 20)
driver.get(URL); time.sleep(2)

results=[]

while True:
    cards = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, CARD)))

    for i in range(len(cards)):
        cards = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, CARD)))
        card = cards[i]

        driver.execute_script("arguments[0].scrollIntoView({block:'center'});", card); time.sleep(0.2)
        start_url = driver.current_url

        # card IS the link
        driver.execute_script("arguments[0].setAttribute('target','_self'); arguments[0].click();", card)

        # ensure we actually navigated
        try: wait.until(lambda d: d.current_url != start_url)
        except: continue

        # detail page
        try: h1 = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, DETAIL_H1)))
        except: 
            driver.get(start_url)
            wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, CARD)))
            continue

        name = (h1.text or "").strip()

        try:
            p = driver.find_element(By.CSS_SELECTOR, AMSTERDAM_P).text or ""
            is_amsterdam = "amsterdam" in p.lower()
        except:
            is_amsterdam = "amsterdam" in (driver.find_element(By.TAG_NAME, "body").text or "").lower()

        if is_amsterdam and name:
            results.append({"Naam": name})

        driver.back()
        wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, CARD)))
        time.sleep(0.5)

    # next page (only after finishing cards)
    nxts = driver.find_elements(By.CSS_SELECTOR, NEXT)
    nxt = next((x for x in nxts if x.is_displayed() and x.is_enabled()), None)
    if not nxt: break

    start_url = driver.current_url
    driver.execute_script("arguments[0].scrollIntoView({block:'center'}); arguments[0].click();", nxt)
    try: wait.until(lambda d: d.current_url != start_url)
    except: pass
    wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, CARD)))
    time.sleep(1)

codesocialeondernemingen = pd.DataFrame(results).drop_duplicates()
driver.quit()


                                Naam
0                   Peer Accountants
1   Stichting Educatief Centrum Oost
2                           Cycle-Up
3                      ROCK Products
4                        Reisbrigade
5                 Roetz Fair Factory
6                            ImPower
7                      Better Places
8                       Everyone.org
9                       Not For Sale
10                  A Beautiful Mess
11                      Moyee Coffee
12            Brouwerij De 7 Deugden
13                        Squarewise
14                      Rederij Kees
15                              CRE8
16                       Money Start
17                        Oma’s Soep


**PSO30**
Ten slotte de PSO30. Hier wordt Amsterdam geselecteerd dan trede 3 en 30+. Parse door de kaarten en sla het adres en de naam op.

In [ ]:
URL = "https://www.pso-nederland.nl/over-pso/gecertificeerde-organisaties?contentType=organisation&distance=10"

d = webdriver.Firefox()
w = WebDriverWait(d, 20)
d.get(URL); time.sleep(2)

# --- FILTERS ---
w.until(EC.element_to_be_clickable((By.CSS_SELECTOR,"button.search-button"))).click()
w.until(EC.element_to_be_clickable((By.ID,"Trede-Trede 3 & 30+"))).click()

inp = w.until(EC.element_to_be_clickable((By.CSS_SELECTOR,"input[role='combobox']")))
inp.clear(); inp.send_keys("Amsterdam"); time.sleep(.8); inp.send_keys(Keys.ENTER)

Select(w.until(EC.presence_of_element_located((By.ID,"range")))).select_by_value("5")
w.until(EC.element_to_be_clickable((By.CSS_SELECTOR,"button.search-button"))).click()
time.sleep(1.5)

# --- COLLECT CARD LINKS (KEY FIX) ---
cards = w.until(EC.presence_of_all_elements_located(
    (By.XPATH,"//a[.//span[normalize-space()='Lees meer']]")
))
links = [c.get_attribute("href") for c in cards if c.get_attribute("href")]

results = []

# --- VISIT EACH DETAIL PAGE DIRECTLY ---
for link in links:
    d.get(link)

    try:
        name = w.until(EC.presence_of_element_located(
            (By.CSS_SELECTOR,"h1.text-2xl"))
        ).text.strip()
    except:
        name = ""

    try:
        parts = d.find_elements(By.XPATH,
            "//dt[normalize-space()='Adres']/following-sibling::dd//div/div")
        address = ", ".join(p.text.strip() for p in parts if p.text.strip())
    except:
        address = ""

    results.append({"Naam": name, "Adres": address})

PSO30 = pd.DataFrame(results)
d.quit()

                                                 Naam  \
0                                         Motopp B.V.   
1                     Peer Accountants Amsterdam B.V.   
2                                   Code Academy B.V.   
3                                               Swink   
4               Stichting Kringloopbedrijf De Lokatie   
5                                De Tweede Jeugd B.V.   
6                                           CRE8 B.V.   
7                                       Ctalents B.V.   
8   PSO en Brookhuis: sociaal ondernemen begint bi...   
9   12,5 jaar PSO: Stichting Supportpunt sinds dag...   
10  Tempo-Team Employability behaalt opnieuw PSO T...   

                                                Adres  
0                 's-gravenhekje 1a, 1011TG Amsterdam  
1                   Baarsjesweg 224, 1058AA Amsterdam  
2                   Magneetstraat 7, 1014CC Amsterdam  
3                 Zekeringstraat 33, 1014BV Amsterdam  
4                      Distelweg 85

In [ ]:
https://companyinfo.nl/freemium